# Assessing the founded Policy Set

In [16]:
# Import general python packages
import pandas as pd
import numpy as np
import copy

# Import functions
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation
from problem_formulation import sum_over,time_step_0,time_step_1, time_step_2, time_step_3, time_step_4

# Loading in the necessary modules for EMA workbench and functions
from ema_workbench import (Model, MultiprocessingEvaluator, Scenario,
                           Constraint, ScalarOutcome, TimeSeriesOutcome, ArrayOutcome)
from ema_workbench.util import ema_logging
from ema_workbench import save_results, load_results
from ema_workbench.em_framework.optimization import (EpsilonProgress)

In [17]:
if __name__ == '__main__':
    ema_logging.log_to_stderr(ema_logging.INFO)

    dike_model, planning_steps = get_model_for_problem_formulation(7)

In [18]:
def problem_formulation_actor(problem_formulation_actor):
   
    # Load the model:
    function = DikeNetwork()
    # workbench model:
    model = Model('dikesnet', function=function)
    # Outcomes are all costs, thus they have to minimized:
    direction = ScalarOutcome.MINIMIZE
    
    model.uncertainties = uncertainties
    model.levers = levers
    
    cost_variables = []
    # cost_variables.extend(
    # [
    #     f"{dike}_{e}"
    #     for e in ["Expected Annual Damage", "Dike Investment Costs"]
    #     for dike in function.dikelist
    # ])
    # cost_variables.extend([f"RfR Total Costs"])
    # cost_variables.extend([f"Expected Evacuation Costs"])


    if problem_formulation_actor == 4: #RWS
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage',
                            variable_name=['{}_Expected Annual Damage'.format(dike)
                                                for dike in function.dikelist],
                            function=sum_over, kind=direction),

            ScalarOutcome('Total Investment Costs',
                            variable_name=['{}_Dike Investment Costs'.format(dike)
                                                for dike in function.dikelist] + ['RfR Total Costs'] + ['Expected Evacuation Costs'],
                            function=sum_over, kind=direction),

            ScalarOutcome('Expected Number of Deaths',
                            variable_name=['{}_Expected Number of Deaths'.format(dike)
                                                for dike in function.dikelist],
                            function=sum_over, kind=direction)]
    
    elif problem_formulation_actor == 5: # GELDERLAND
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage A1', variable_name='A.1_Expected Annual Damage', function = sum_over, kind=direction),
            ScalarOutcome('Expected Annual Damage A2', variable_name='A.2_Expected Annual Damage', function = sum_over, kind=direction),
            ScalarOutcome('Expected Annual Damage A3', variable_name='A.3_Expected Annual Damage', function = sum_over, kind=direction),
            ScalarOutcome('Expected Number of Deaths in A1', variable_name='A.1_Expected Number of Deaths',function = sum_over, kind=direction),
            ScalarOutcome('Expected Number of Deaths in A2', variable_name='A.2_Expected Number of Deaths',function = sum_over, kind=direction),
            ScalarOutcome('Expected Number of Deaths in A3', variable_name='A.3_Expected Number of Deaths',function = sum_over, kind=direction),
            ScalarOutcome('Total Costs', variable_name=cost_variables, function = sum_over, kind=direction),
            ScalarOutcome('Aggregated Expected Number of Deaths A4-A5', variable_name = 
            ['A.4_Expected Number of Deaths', 'A.5_Expected Number of Deaths'], function = sum_over, kind=direction),
            ]

    elif problem_formulation_actor == 6: #OVERIJSSEL
        model.outcomes.clear()
        
        model.outcomes = [
            ScalarOutcome(f'Total_period_Costs 0', variable_name= dike_model.outcomes['Total_period_Costs'].variable_name, function = time_step_0, kind=direction),
            ScalarOutcome(f'Total_period_Costs 1', variable_name= dike_model.outcomes['Total_period_Costs'].variable_name, function = time_step_1, kind=direction),
            ScalarOutcome(f'Total_period_Costs 2', variable_name= dike_model.outcomes['Total_period_Costs'].variable_name, function = time_step_2, kind=direction),
            ScalarOutcome(f'Total_period_Costs 3', variable_name= dike_model.outcomes['Total_period_Costs'].variable_name, function = time_step_3, kind=direction),
            ScalarOutcome(f'Total_period_Costs 4', variable_name= dike_model.outcomes['Total_period_Costs'].variable_name, function = time_step_4, kind=direction),
            ScalarOutcome("Expected Number of Deaths_", variable_name =
            [f"{dike}_Expected Number of Deaths_" for dike in function.dikelist], function = sum_over, kind=direction),
            ScalarOutcome('Expected Annual Damage A4_', variable_name='A.1_Expected Annual Damage', function = sum_over, kind=direction),
            ScalarOutcome('Expected Annual Damage A5_', variable_name='A.1_Expected Annual Damage', function = sum_over, kind=direction)
        ]
    else:
        raise TypeError('unknown identifier')
        
    return model

In [20]:
uncertainties = dike_model.uncertainties
uncertainties = copy.deepcopy(dike_model.uncertainties)

In [21]:
levers = dike_model.levers
levers = copy.deepcopy(dike_model.levers)

In [22]:
model = problem_formulation_actor(6)


In [23]:
#read a CSV file into a DataFrame
rf_policy_set= pd.read_csv('Week24_MORDM_outcomes_overijssel_PD6 (1).csv')

In [24]:
rf_policy_set

,0_RfR 0,0_RfR 1,0_RfR 2,0_RfR 3,0_RfR 4,1_RfR 0,1_RfR 1,1_RfR 2,1_RfR 3,1_RfR 4,...,A.5_DikeIncrease 1,A.5_DikeIncrease 2,A.5_DikeIncrease 3,A.5_DikeIncrease 4,Expected Annual Damage A4,Expected Annual Damage A5,Expected Number of Deaths in A4,Expected Number of Deaths in A5,Total Costs,Aggregated Expected Number of Deaths A1-A3
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.481009e+08,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.481009e+08,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.481009e+08,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.481009e+08,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.481009e+08,0.0


In [25]:
from ema_workbench.analysis import parcoords

ref_data = rf_policy_set.loc[:, [o.name for o in model.outcomes]]
ref_limits = parcoords.get_limits(ref_data)
ref_limits.loc[0, ['Expected Annual Damage A4', 'Expected Annual Damage A5','Expected Number of Deaths in A5', 'Expected Number of Deaths in A4', 'Total Costs']] = 0

paraxes = parcoords.ParallelAxes(ref_limits)
paraxes.plot(ref_data)
plt.title('Trade-offs in Reference case scenario')

# Set figure size
# Run this twice, to get a nice broad view
fig_size = plt.rcParams["figure.figsize"]

fig_size[0] = 12
fig_size[1] = 12
plt.rcParams["figure.figsize"] = fig_size
plt.show()


KeyError: "None of [Index(['Total_period_Costs 0', 'Total_period_Costs 1', 'Total_period_Costs 2',\n       'Total_period_Costs 3', 'Total_period_Costs 4',\n       'Expected Number of Deaths_', 'Expected Annual Damage A4_',\n       'Expected Annual Damage A5_'],\n      dtype='object')] are in the [columns]"